<a href="https://colab.research.google.com/github/edyoda/AI-Agent-Development-and-GenAI/blob/main/Day5_Agent_AI_Development_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ["SERPAPI_API_KEY"] = userdata.get("SERPAPI_API_KEY")

In [5]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00


In [8]:
!pip install duckduckgo-search -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.0 MB/s eta 0:00:00


#1. Goal-oriented AI agent follows a structured objective, like answering questions, retrieving data, or executing tasks.

In [9]:
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType
from langchain.tools import DuckDuckGoSearchRun # Import the DuckDuckGoSearchRun tool

llm = ChatOpenAI(model="gpt-4", temperature=0)

# Create a list of tools, for example, using DuckDuckGoSearchRun
tools = [DuckDuckGoSearchRun()]

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

response = agent.run("What is the capital of France?")
print(response)

<ipython-input-9-158da415fee7>:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("What is the capital of France?")




> Entering new AgentExecutor chain...
I know this information. The capital of France is Paris.
Final Answer: The capital of France is Paris.

> Finished chain.
The capital of France is Paris.


In [11]:
response = agent.run("What can you not do?")
print(response)



> Entering new AgentExecutor chain...
This question is quite broad and could refer to many things. It could be asking about physical limitations, legal restrictions, or even philosophical concepts. However, as an AI, I have specific limitations. 
Final Answer: As an AI, I cannot perform physical tasks, feel emotions, or make subjective judgments. I also cannot access personal data unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality.

> Finished chain.
As an AI, I cannot perform physical tasks, feel emotions, or make subjective judgments. I also cannot access personal data unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality.


#2. Using Tools & Plugins for Agent Expansion

In [13]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=99d6459767fb80fa881fe163c62c2c223007cf0dd2c4041d6110f498fe5152f2
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [14]:
from langchain.tools import Tool
from langchain.utilities import WikipediaAPIWrapper

wikipedia = WikipediaAPIWrapper()
wiki_tool = Tool(
    name="Wikipedia Search",
    func=wikipedia.run,
    description="Search Wikipedia for information."
)

agent = initialize_agent([wiki_tool], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

response = agent.run("Tell me about Quantum Mechanics using Wikipedia.")
print(response)




> Entering new AgentExecutor chain...
I need to search for Quantum Mechanics on Wikipedia to gather information.
Action: Wikipedia Search
Action Input: Quantum Mechanics
Observation: Page: Quantum mechanics
Summary: Quantum mechanics is the fundamental physical theory that describes the behavior of matter and of light; its unusual characteristics typically occur at and below the scale of atoms.: 1.1  It is the foundation of all quantum physics, which includes quantum chemistry, quantum field theory, quantum technology, and quantum information science.
Quantum mechanics can describe many systems that classical physics cannot. Classical physics can describe many aspects of nature at an ordinary (macroscopic and (optical) microscopic) scale, but is not sufficient for describing them at very small submicroscopic (atomic and subatomic) scales. Classical mechanics can be derived from quantum mechanics as an approximation that is valid at ordinary scales.
Quantum systems have bound states t

# 3. Implementing Multi-Step Reasoning

In [19]:
# @title Default title text
from langchain.agents import load_tools

tools = load_tools(["serpapi", "llm-math"], llm=llm)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

response = agent.run("What is the square root of the population of Japan?")
print(response)




> Entering new AgentExecutor chain...
First, I need to find the current population of Japan.
Action: Search
Action Input: "current population of Japan"
Observation: {'type': 'population_result', 'population': '124.5 million', 'year': '2023'}
Thought:The current population of Japan is 124.5 million. Now I need to convert this to a number and find the square root.
Action: Calculator
Action Input: sqrt(124500000)
Observation: Answer: 11157.956802210698
Thought:I have calculated the square root of the population of Japan.
Final Answer: The square root of the population of Japan is approximately 11158.

> Finished chain.
The square root of the population of Japan is approximately 11158.


In [21]:
def custom_input_handler(user_input):
    if "weather" in user_input.lower():
        return "I can't fetch live weather data, but you can check on weather.com"
    return agent.run(user_input)

print(custom_input_handler("What is the weather in New York?"))
print(custom_input_handler("Who was Albert Einstein?"))

I can't fetch live weather data, but you can check on weather.com


> Entering new AgentExecutor chain...
Albert Einstein was a famous physicist, but to provide a detailed answer, I should look up more information.
Action: Search
Action Input: "Albert Einstein"
Observation: ["Albert Einstein was a German-born theoretical physicist and mathematician who developed the theory of relativity and made significant contributions to quantum mechanics. His mass–energy equivalence formula, E = mc², is considered “the world's most famous equation”. Einstein's theories and vision of physics changed how people understand the universe. Although controversial at the time, his work overturned Newtonian gravitational theory.", 'Albert Einstein type: Theoretical physicist.', 'Albert Einstein entity_type: people.', 'Albert Einstein main_tab_text: Overview.', 'Albert Einstein kgmid: /m/0jcx.', 'Albert Einstein born: March 14, 1879, Ulm, Germany.', 'Albert Einstein died: April 18, 1955 (age 76 years), Princ

In [22]:
import logging

logging.basicConfig(level=logging.INFO)
agent = initialize_agent([wiki_tool], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

response = agent.run("Tell me about relativity in simple terms.")
print(response)




> Entering new AgentExecutor chain...
I need to find a simple explanation of the concept of relativity. I will use Wikipedia to find this information.
Action: Wikipedia Search
Action Input: Relativity
Observation: Page: Theory of relativity
Summary: The theory of relativity usually encompasses two interrelated physics theories by Albert Einstein: special relativity and general relativity, proposed and published in 1905 and 1915, respectively. Special relativity applies to all physical phenomena in the absence of gravity. General relativity explains the law of gravitation and its relation to the forces of nature. It applies to the cosmological and astrophysical realm, including astronomy.
The theory transformed theoretical physics and astronomy during the 20th century, superseding a 200-year-old theory of mechanics created primarily by Isaac Newton. It introduced concepts including 4-dimensional spacetime as a unified entity of space and time, relativity of simultaneity, kinematic and

In [24]:
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.3 MB/s eta 0:00:00


In [25]:
!pip install uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.6 MB/s eta 0:00:00
